In [37]:
import pandas as pd

In [38]:
df = pd.read_csv('dataset/results.csv')

In [27]:
df.describe()

,home_score,away_score
count,44557.000000,44557.000000
mean,1.739278,1.178401
std,1.747263,1.393775
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000


In [39]:
sample = df[(df.tournament == 'FIFA World Cup') & (df.country == 'Qatar') & ((df.home_team == 'Senegal')|(df.away_team == 'Senegal'))].copy()

def get_result(row,side):
    if side == 'home':
        if row.home_score > row.away_score:
            return 3
        elif row.home_score < row.away_score:
            return 0
        else:
            return 1
    else:
        if row.home_score < row.away_score:
            return 3
        elif row.home_score > row.away_score:
            return 0
        else:
            return 1

sample['home_result'] = sample.apply(lambda x: get_result(x,'home'),axis=1)
sample['away_result'] = sample.apply(lambda x: get_result(x,'away'),axis=1)

home = sample[['date','home_team','home_result']].copy()
away = sample[['date','away_team','away_result']].copy()

home.rename(columns = {'home_team':'team','home_result':'result'}, inplace=True)
away.rename(columns = {'away_team':'team','away_result':'result'}, inplace=True)


In [41]:
pd.concat([home,away])

,date,team,result
44290,2022-11-21,Senegal,0
44305,2022-11-25,Qatar,0
44321,2022-11-29,Ecuador,0
44340,2022-12-04,England,3
44290,2022-11-21,Netherlands,3
44305,2022-11-25,Senegal,3
44321,2022-11-29,Senegal,3
44340,2022-12-04,Senegal,0


**Next Steps**

Look at the number of games. Since group stage is 3 matches, 3 means eliminated at group stage, so the ranking is between 17th and 32nd. If 4 matches, then it would be round of 16 (so ranking would be between 9th and 16th), and so on. Within each results, we can sort based on points accumulated (and goal scored) <== have to add that later.